In [16]:
%pwd

'd:\\ml\\end_to_end_airbnb_machine_learning_project\\Seattle-airbnb-house-price-prediction-with-mlflow-and-aws-deployment'

In [17]:
import os
os.chdir("d:\\ml\\end_to_end_airbnb_machine_learning_project\\Seattle-airbnb-house-price-prediction-with-mlflow-and-aws-deployment")
%pwd

'd:\\ml\\end_to_end_airbnb_machine_learning_project\\Seattle-airbnb-house-price-prediction-with-mlflow-and-aws-deployment'

In [18]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    

In [19]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [20]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

        

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )

        return data_transformation_config

In [21]:
import os
from mlProject import logger
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import joblib


In [22]:
class DataTransformation:
    def __init__(self, config):
        self.config = config

    def train_test_spliting(self):
        # Load data
        data = pd.read_csv(self.config.data_path)

        # Drop the first two columns
        data = data.drop(columns=data.columns[:2])  # Dropping the first two columns

        # Encode categorical features
        categorical_columns = data.select_dtypes(include=['object']).columns  # Adjust if needed
        label_encoders = {}
        for column in categorical_columns:
            le = LabelEncoder()
            data[column] = le.fit_transform(data[column].astype(str))
            label_encoders[column] = le  # Save encoder for later use

        # Save encoders using joblib for future use
        encoder_save_path = os.path.join(self.config.root_dir, "label_encoders.joblib")
        joblib.dump(label_encoders, encoder_save_path)
        logger.info(f"Saved label encoders to {encoder_save_path}")

        logger.info("Encoded categorical features")
        logger.info(f"Categorical columns: {list(categorical_columns)}")

        # Split into train and test
        train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

        # Save to CSV
        train_data.to_csv(os.path.join(self.config.root_dir, "train.csv"), index=False)
        test_data.to_csv(os.path.join(self.config.root_dir, "test.csv"), index=False)

        logger.info("Split data into train and test")
        logger.info(f"Train data: {train_data.shape}")
        logger.info(f"Test data: {test_data.shape}")


In [23]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_spliting()
except Exception as e:
    raise e

[2024-12-04 15:41:48,926: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-04 15:41:48,930: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-04 15:41:48,935: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-12-04 15:41:48,938: INFO: common: created directory at: artifacts]
[2024-12-04 15:41:48,940: INFO: common: created directory at: artifacts/data_transformation]
[2024-12-04 15:41:48,996: INFO: 3947289495: Saved label encoders to artifacts/data_transformation\label_encoders.joblib]
[2024-12-04 15:41:48,997: INFO: 3947289495: Encoded categorical features]
[2024-12-04 15:41:48,997: INFO: 3947289495: Categorical columns: ['fuel', 'seller_type', 'transmission', 'owner']]
[2024-12-04 15:41:49,049: INFO: 3947289495: Split data into train and test]
[2024-12-04 15:41:49,049: INFO: 3947289495: Train data: (6502, 11)]
[2024-12-04 15:41:49,049: INFO: 3947289495: Test data: (1626, 11)]
